In [9]:
import os
import json
import pandas as pd

In [2]:
dataset_grid_res = {x.split('_')[0]: x for x in os.listdir('grid_search_results') if
                    x.endswith('.csv') and 'process' not in x}
dataset_grid_res

{'accent': 'accent_grid_val.csv',
 'ecoli': 'ecoli_grid_val.csv',
 'wifi': 'wifi_grid_val.csv',
 'banknote': 'banknote_grid_val.csv',
 'iris': 'iris_grid_val.csv',
 'parkinson': 'parkinson_grid_val.csv',
 'wine': 'wine_grid_val.csv'}

In [3]:
os.makedirs("param_files", exist_ok=True)

In [8]:
for dataset, grid_res in dataset_grid_res.items():
    df = pd.read_csv(os.path.join('grid_search_results', grid_res))
    df["partial_scores_mean"] = df["partial_score"].apply(lambda x: float(x.split(" +- ")[0]))
    df["diff_full_partial"] = df["full_score"] - df["partial_scores_mean"]
    df.sort_values(by="diff_full_partial", ascending=True, inplace=True, kind='stable')
    df.sort_values(by="partial_scores_mean", ascending=False, inplace=True, kind='stable')
    df.to_csv(os.path.join("param_files", f"{dataset}_params.csv"), index=False)


    # create params dict of reg_type	weight_type	alpha	use_layer_norm	use_aug	lr	batch_size

    params = {
        "reg_type": str(df.iloc[0]["reg_type"]),
        "weight_type": str(df.iloc[0]["weight_type"]),
        "alpha": float(df.iloc[0]["alpha"]),
        "use_layer_norm": bool(df.iloc[0]["use_layer_norm"]),
        "use_aug": bool(df.iloc[0]["use_aug"]),
        "lr": float(df.iloc[0]["lr"]),
        "batch_size": int(df.iloc[0]["batch_size"]),
    }

    with open(os.path.join("param_files", f"{dataset}_params.json"), 'w') as f:
        json.dump(params, f)


In [5]:
for i, dataset in enumerate(os.listdir("param_files")):
    if dataset.endswith(".json"):
        f = open("removal_test_commands", 'w' if i == 0 else 'a')
        dataset = dataset.split("_")[0]
        f.write(f"python main_experimens.py --dataset {dataset.capitalize()} --run_grid False --run_ts False\n")

In [7]:
# load all params files to one df

df = pd.DataFrame()

for dataset in os.listdir("param_files"):
    if dataset.endswith(".json"):
        dataset = dataset.split("_")[0]

        if dataset.lower() in ['wifi', 'wine']:
            continue

        with open(os.path.join("param_files", dataset + "_params.json"), 'r') as f:
            params = json.load(f)
        df = df.append(pd.DataFrame(params, index=[dataset]))

df

,reg_type,weight_type,alpha,use_layer_norm,use_aug,lr,batch_size
ecoli,l2,mult,1.0,True,True,0.010,32
banknote,max,loss,0.0,True,True,0.100,64
iris,l1,None,0.0,False,True,0.100,64
parkinson,l1,None,1.0,False,True,0.001,64
accent,l2,None,0.5,False,True,0.100,32
